In [134]:
"""
A filesystem cache of previous queries to Geocod.io, so that we don't waste too many queries
(I only get 2500 free queries per day).
Assumes that the project root (or wherever you started your python shell/Jupyter notebook from) has
a file "geocodio_api_key.json" with the contents:
    {
        "key": "API_KEY_HERE"
    }
"""
import json
import os
from pathlib import Path
from typing import Dict, Iterable, List
from geocodio import GeocodioClient
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

# A cache of format "address -> Geocode response dict" that we will keep committed
# in the repo.
CACHE_PATH = Path('./data/geocode_cache.json')

def get_client():
    return GeocodioClient(json.loads(Path('./geocodio_api_key.json').read_text())['key'])

def load_cache() -> Dict[str, dict]:
    if not CACHE_PATH.exists():
        return {}
    with CACHE_PATH.open() as f:
        return json.load(f)

def overwrite_cache(cache: Dict[str, dict]) -> None:
    with CACHE_PATH.open('w') as f:
        json.dump(cache, f)

def lookup(addresses: Iterable[str]) -> List[dict]:
    """
    Please don't run this function in parallel, because the cache isn't thread-safe.
    Since it takes an Iterable, it's easy to use this with a Pandas series:
        df['geocode_results'] = geocode_cache.lookup(df['address'])
    """
    cache = load_cache()
    addresses_to_lookup = list(set(addresses) - set(cache.keys()))

    if len(addresses_to_lookup):
        api_results = get_client().geocode(addresses_to_lookup)
        for address, response in zip(addresses_to_lookup, api_results):
            cache[address] = dict(response)

        overwrite_cache(cache)

    return [cache[address] for address in addresses]

In [135]:
results = lookup(['137  Fair Oaks St, Mountain View, CA'])
points = [Point(r['results'][0]['location']['lat'], r['results'][0]['location']['lng']) for r in results]

In [136]:
newer_permits = pd.read_csv('./data/APRs.csv')

In [137]:
newer_permits.shape

(620, 48)

In [138]:
newer_permits['Address'] = newer_permits['Address'].str.title() + ', Mountain View, CA'

In [139]:
newer_permits['Address']

0             137  Fair Oaks St, Mountain View, CA
1           285 Carmelita Drive, Mountain View, CA
2                423  Loreto St, Mountain View, CA
3             125  Fair Oaks St, Mountain View, CA
4      545 Mountain View Avenue, Mountain View, CA
                          ...                     
615             333  Apricot Ln, Mountain View, CA
616         257 Calderon Avenue, Mountain View, CA
617     315 Sierra Vista Avenue, Mountain View, CA
618             351  Martens Av, Mountain View, CA
619           135  Fair Oaks St, Mountain View, CA
Name: Address, Length: 620, dtype: object

In [140]:
all_results = lookup(newer_permits['Address'])

In [141]:
def geocode_results_to_geoseries(georesults, df):
    return gpd.GeoSeries([geocode_result_to_point(result) for result in georesults], index=df.index)

In [142]:
def geocode_result_to_point(georesult):
    if not georesult.get('results'):
        return np.nan
    loc = georesult['results'][0]['location']
    return Point(loc['lng'], loc['lat'])

In [143]:
newer_permits = gpd.GeoDataFrame(newer_permits, 
                                 geometry=geocode_results_to_geoseries(all_results, newer_permits), 
                                 crs='EPSG:4326')

In [144]:
newer_permits.to_file("./data/newer_permits.json", driver="GeoJSON")

In [145]:
newer_permits.shape

(620, 49)

In [146]:
ABAG = None
def load_abag_permits() -> gpd.GeoDataFrame:
    """
    Loads all 2013-2017 building permits from ABAG as a GeoDataFrame.
    """
    global ABAG
    if ABAG is None:
        geometry_df = gpd.read_file("./data/abag_permits/permits.shp")
        data_df = pd.read_csv("./data/abag_permits/permits.csv")

        # There shouldn't be any rows with geometry data that don't have label data
        assert geometry_df["joinid"].isin(data_df["joinid"]).all()

        ABAG = gpd.GeoDataFrame(data_df.merge(geometry_df, how="left", on="joinid"))

    # Filter out permits from before the start of the 5th Housing Element cycle.
    ABAG = ABAG[ABAG['permyear'] >= 2015].copy()

    ABAG['apn'] = ABAG['apn'].replace({np.nan: None})

    return ABAG

In [147]:
abag_perms = load_abag_permits()
older_permits = abag_perms[(abag_perms['jurisdictn'] == 'Mountain View')]

In [148]:
older_permits.columns

Index(['objectid', 'joinid', 'permyear', 'county', 'jurisdictn', 'apn',
       'address', 'zip', 'projname', 'hcategory', 'vlowdr', 'vlowndr',
       'vlowtot', 'lowdr', 'lowndr', 'lowtot', 'moddr', 'modndr', 'modtot',
       'amodtot', 'totalunit', 'tenure', 'mapped', 'mapnotes', 'pda',
       'pdacycle', 'pdajoinid', 'tpa', 'tpacycle', 'hsngsite', 'rhnacycle',
       'notes', 'ObjectId2', 'OBJECTID', 'geometry'],
      dtype='object')

In [149]:
# Remove constant columns
older_permits = older_permits.loc[:,~(older_permits == older_permits.iloc[0]).all()]

In [150]:
# Remove columns with no real values
older_permits = older_permits.iloc[:, ~older_permits.isna().all().values]

In [152]:
older_permits = older_permits.copy()

In [154]:
older_permits.columns

Index(['joinid', 'permyear', 'apn', 'address', 'zip', 'projname', 'hcategory',
       'vlowdr', 'vlowtot', 'lowdr', 'lowtot', 'amodtot', 'totalunit',
       'tenure', 'mapnotes', 'pda', 'pdacycle', 'pdajoinid', 'tpa', 'tpacycle',
       'hsngsite', 'rhnacycle', 'notes', 'ObjectId2', 'OBJECTID', 'geometry'],
      dtype='object')

In [155]:
older_permits.drop(['joinid',
                    'zip',
                    'pda', 
                    'pdacycle', 
                    'pdajoinid',
                    'tpa',
                    'tpacycle', 
                    'rhnacycle',
                    'ObjectId2',
                    'OBJECTID',
                    'mapnotes',
                    'notes'], axis=1, inplace=True)

In [214]:
older_permits.columns

Index(['permyear', 'apn', 'address', 'projname', 'hcategory', 'vlowdr',
       'vlowtot', 'lowdr', 'lowtot', 'amodtot', 'totalunit', 'tenure',
       'hsngsite', 'geometry'],
      dtype='object')

In [156]:
newer_permits = newer_permits.iloc[:, ~newer_permits.isna().all().values]

In [157]:
newer_permits = newer_permits.loc[:,~(newer_permits == newer_permits.iloc[0]).all()]

In [158]:
newer_permits.columns

Index(['Reporting Year', 'Prior APN', 'APN', 'Address', 'Project Name',
       'Jurisdiction Tracking ID', 'Unit Category', 'Tenure',
       'Entitled Very Low Income Deed (Restricted)',
       'Entitled Low Income Deed (Restricted)',
       'Entitled Moderate Income Deed (Restricted)',
       'Entitled Above Moderate Income', 'Entitlement Date Approved',
       'Total Units Entitled', 'Permitted Very Low Income Deed (Restricted)',
       'Permitted Low Income Deed (Restricted)',
       'Permitted Moderate Income Deed (Restricted)',
       'Permitted Above Moderate Income', 'Permit Date Issued',
       'Total  Units Permitted', 'Completed Very Low Income Deed (Restricted)',
       'Completed  Low Income Deed (Restricted)',
       'Completed  Above Moderate Income', 'Completed Date',
       'Total Units Completed', 'Total Extremely Low Income Units',
       'SB 35 Approved?', 'Financial Assistance Program',
       'Deed Restriction Type', 'Years of Affordability',
       'Demolished/Des

In [ ]:
newer_permits['vlowtot'] = newer_permits['Permitted Very Low Income Deed (Restricted)'] + newer_permits['Permitted Very Low Income Deed (Restricted)']

In [242]:
newer_permits.rename({'Reporting Year': 'permyear',
                      'APN': 'apn',
                      'Address': 'address',
                      'Project Name': 'projname',
                      'Unit Category': 'hcategory',
                      'Tenure': 'tenure',
                      'Notes': 'notes',
                      'Permitted Very Low Income Deed (Restricted)': 'vlowdr',
                      'Permitted Low Income Deed (Restricted)': 'lowdr',
                      'Permitted Above Moderate Income': 'amodtot',
                      'Total\xa0 Units Permitted': 'totalunit',
                       }, axis=1, inplace=True)

In [248]:
newer_permits = newer_permits[[c for c in newer_permits.columns if c in older_permits.columns]]

In [249]:
newer_permits.columns

Index(['permyear', 'apn', 'address', 'projname', 'hcategory', 'tenure',
       'vlowdr', 'lowdr', 'amodtot', 'totalunit', 'geometry'],
      dtype='object')

In [270]:
newer_permits.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [271]:
older_permits.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [251]:
permits = pd.concat((newer_permits, older_permits), axis=0)

In [253]:
permits['tenure'].value_counts()

Owner     537
O         168
Renter     83
R           6
Name: tenure, dtype: int64

In [257]:
permits['tenure'].replace({'O': "Owner", 'R': "Renter"}, inplace=True)

In [258]:
permits['tenure'].value_counts()

Owner     705
Renter     89
Name: tenure, dtype: int64

In [259]:
permits['hcategory'].value_counts()

Single-Family Attached Unit               390
5+                                        256
SF                                        148
Accessory Dwelling Unit                   105
2 to 4                                     75
Single-Family Detached Unit                66
5 or More Units Per Structure              57
SU                                          7
2-, 3-, and 4-Plex Units per Structure      2
Name: hcategory, dtype: int64

In [263]:
permits['hcategory'].replace({'5+': "5 or More Units Per Structure", 
                              '2 to 4': "2-, 3-, and 4-Plex Units per Structure"}, inplace=True)

I don't know what to do with SU or SF.

In [264]:
permits['hcategory'].value_counts()

Single-Family Attached Unit               390
5 or More Units Per Structure             313
SF                                        148
Accessory Dwelling Unit                   105
2-, 3-, and 4-Plex Units per Structure     77
Single-Family Detached Unit                66
SU                                          7
Name: hcategory, dtype: int64

In [269]:
permits.to_file('./data/all_permits.json', index=False)